In [ ]:
# Mohammad Ghadban - 20.05-2019
'''
    First step here is to download and import the nessesary 
    libraries such as follows:
    
    1. importing gzip and zipfile for the compressing and decompressing
    of the downloaded files.
    
    2. importing gensim for unsupervised semantic 
    modelling from plain text.
    
    3. importin the logging to watch the flow in the development
    process.
    
'''
#!pip3 install -qU gensim==3.6.0
import gzip #compress and decompress
from zipfile import ZipFile #compress and decompress
import gensim #unsupervised semantic modelling from plain text
import logging #showing the flow of development 
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

In [ ]:
# specifying the zip file name 
file_name = "data_downloaded/word-embeddings.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with open('glove_emb.pkl', 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")

In [ ]:
'''
    Understanding some of the parameters:
    To train the model, we have to set some parameters. 
    Now, let's try to understand what some of them mean. 
    For reference, this is the command that we used to train the model.
    
    Size:
    the dense vector to represent each toke or word.
    If you have limited data then the size should be a much smaller value.
    If you have lots of data, its good to experiment with various sizes.
    A value of 100 - 150 might work well, but you need to experiment.
    
    Window:
    The maximum distance between 
    the target word and its neighboring word.
    If your neighbor's position
    is greater than the maximum window width to the left and the right, 
    then, some neighbors are not considered
    as being related to the target word. 
    In theory, a smaller window 
    should give you terms that are more related. 
    If you have lots of data,
    then the window size should not matter too much, 
    as long as its a decent sized window
    
    
    min_count:
    Minimium frequency count of words.
    The model would ignore words that do not statisfy the min_count.
    Extremely infrequent words are usually unimportant,
    so its best to get rid of those. 
    Unless your dataset is really tiny,
    this does not really affect the model.

    
    Workers:
    How many threads to use behind the scenes?
    
'''
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)



In [ ]:
#we are saving the model in i abinary file and a txt file
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.models import Word2Vec, KeyedVectors   
model.wv.save_word2vec_format('model.bin', binary=True)
model.wv.save_word2vec_format('model.txt', binary=False)

In [ ]:
#load the model
wv_from_text = KeyedVectors.load_word2vec_format(
    ('model.txt'), 
    binary=False)  # C text format

In [ ]:
'''Now lets look at some outout:
    This first example shows a simple case 
    of looking up words similar to the word dirty. 
    All we need to do here is to call the most_similar function
    and provide the word dirty as the positive example. 
    This returns the top 10 similar words.


    Example
    ---------------------------
''' 

w1 = 'dirty'
wv_from_text.most_similar(positive=w1)

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["car"]
wv_from_text.most_similar (positive=w1,topn=6)

In [ ]:
# look up top 6 words similar to 'shocked'
w1 = ["politics"]
wv_from_text.most_similar (positive=w1,topn=6)

In [ ]:
# get everything related to stuff on the bed
w1 = ["weather",'sun','blue']
w2 = ['cold']
wv_from_text.most_similar (positive=w1,negative=w2,topn=10)

In [ ]:
# similarity between two different words
wv_from_text.similarity(w1="rain",w2="cloud")

In [ ]:
# similarity between two identical words
wv_from_text.similarity(w1="dirty",w2="dirty")

In [ ]:
#################################################################

In [ ]:
# TextRank Summarizer

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import networkx as nx
nltk.download('punkt') # one time execution
import re

In [ ]:
#preparing the article 
file = open('data_downloaded/nature_moon.txt', 'r').read()
file =   ""+ file + ""
file = file.split('.')

In [ ]:
'''
Creating a table with columns article_text and article_id
and showing the first 5 rows 
'''
df = pd.DataFrame(file, columns=['article_text'])
df['article_id'] = range(len(df['article_text']))
print('The first five rows:')
df.head()

In [ ]:
# split the text in the articles into sentences

#creating an empty list 
sentences = []

for s in df['article_text']:
    sentences.append(sent_tokenize(s))

# flatten the list
sentences = [y for x in sentences for y in x]

#Showing the first 10 sentences 
#sentences[1:10]
df.head()
print(sentences[1:5])

#showing the length of the list og sentences
print('\n\nlength of sentences: ',len(sentences))

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

print('CLean text:\n')
clean_sentences[1:5]

In [ ]:
nltk.download('stopwords')# one time execution

In [ ]:
#using the english stopwords
stop_words = stopwords.words('english')

In [ ]:
#function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

print(clean_sentences[1:5])

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('model.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((150,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((150,))
    sentence_vectors.append(v)

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,150), sentence_vectors[j].reshape(1,150))[0,0]

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
# Specify number of sentences to form the summary
sn = 1
# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])

In [ ]:
# Specify number of sentences to form the summary
sn = 5
textdone = []
# Generate summary
for i in range(sn):
    textdone.append(ranked_sentences[i][1])

print('Summary:')
for t in textdone:
    print((t))

# When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary.

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work.


#### Finally we could chose other approaches regargind the initializing word-embeddings such as follows:
- Choosing other word2vec or word embediddng data sets
- Making own word2vec/ word-embedding data sets
- The file choosen for this code in my openion is way too big for such asimple task.
    
Have a good one :)